# Title

In [2]:
!pip list | grep "fastai"

fastai                                     2.5.0
You should consider upgrading via the '/home/stormbird/miniconda3/bin/python -m pip install --upgrade pip' command.


In [3]:
from fastai.vision.all import *

In [4]:
# download and untar PASCAL 2007 dataset
path = untar_data(URLs.PASCAL_2007)
path.ls()

KeyboardInterrupt: 

In [5]:
!ls

Makefile  multilabel-classification.ipynb  README.md  requirements.txt


In [ ]:
df = pd.read_csv(path/'train.csv')
df.head()

In [ ]:
dls = ImageDataLoaders.from_df(df, path, folder='train', valid_col='is_valid', label_delim=' ',
                               item_tfms=Resize(460), batch_tfms=aug_transforms(size=224))

In [ ]:
dls.show_batch()

In [ ]:
f1_macro = F1ScoreMulti(thresh=0.5, average='macro')
f1_macro.name = 'F1(macro)'
f1_samples = F1ScoreMulti(thresh=0.5, average='samples')
f1_samples.name = 'F1(samples)'
learn = vision_learner(dls, resnet50, metrics=[partial(accuracy_multi, thresh=0.5), f1_macro, f1_samples])

In [ ]:
learn.lr_find()

In [ ]:
learn.fine_tune(2, 3e-2)

In [ ]:
learn.show_results()

In [ ]:
interp = Interpretation.from_learner(learn)
interp.plot_top_losses(9)